<a href="https://colab.research.google.com/github/shxnyoung/MLDL-woundimage/blob/main/U_net_testmodel_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
import cv2
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Add, LeakyReLU, UpSampling2D
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.callbacks import ModelCheckpoint, TensorBoard
from PIL import Image
from keras.callbacks import ReduceLROnPlateau #adaptive LR
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [ ]:
def datacutter(data, l0, l1, l2):
  train0 = data[:l0,]
  train1 = data[l0:l0+l1,]
  train2 = data[l1:l1+l2,]
  return train0, train1, train2

In [ ]:
x_train=np.load('/content/drive/My Drive/MLDL_models/ClassificationTEST/X.npy')/255.

In [ ]:
l0=730; l1=510; l2=850
x_train0, x_train1, x_train2 = datacutter(x_train, l0, l1, l2)

In [ ]:
class_names= ['granulation', 'slough', 'necrosis']
cy_train0 = np.full((len(x_train0)), 0); cy_train0=tf.keras.utils.to_categorical(cy_train0, num_classes=3)
cy_train1 = np.full((len(x_train1)), 1); cy_train1=tf.keras.utils.to_categorical(cy_train1, num_classes=3)
cy_train2 = np.full((len(x_train2)), 2); cy_train2=tf.keras.utils.to_categorical(cy_train2, num_classes=3)
cy_train = np.concatenate((cy_train0, cy_train1, cy_train2))
cy_train.shape

(2090, 3)

In [ ]:
sy_train=np.load('/content/drive/My Drive/MLDL_models/ClassificationTEST/Y.npy')
sy_train0, sy_train1, sy_train2 = datacutter(sy_train, l0, l1, l2)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, cy_train, cy_val = train_test_split(x_train,cy_train, test_size=0.1, random_state=1004)
sy_train, sy_val = train_test_split(sy_train, test_size=0.1, random_state=1004)

In [ ]:
def compile_and_train_multioutput(multi_output_model, num_epochs): 
    precision = tf.keras.metrics.Precision()
    recall = tf.keras.metrics.Recall()
    model.compile(loss={'co' : 'categorical_crossentropy', 'so' : 'binary_crossentropy'}, optimizer='adam', metrics=['acc']) 
    filepath = '/content/drive/MyDrive/MLDL_models/weights2/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', save_freq=10)
    callback_param = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto', min_lr=1e-05)
    history = model.fit(x_train
                        , {'co' : cy_train, 'so' : sy_train}
                        , validation_data=(x_val, {'co' : cy_val, 'so' : sy_val})
                        , epochs=num_epochs, batch_size=100
                        , callbacks=[callback_param, checkpoint]
                        )
    return history

In [ ]:
def compile_and_train(model, num_epochs): 
    precision = tf.keras.metrics.Precision()
    recall = tf.keras.metrics.Recall()
    model.compile(loss={'categorical_crossentropy', optimizer='adam', metrics=['acc', precision, recall]) 
    filepath = '/content/drive/MyDrive/MLDL_models/weights1/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', save_freq=10)
    callback_param = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto', min_lr=1e-05)
    history = model.fit(x_train, y_train
                        , validation_data=(x_val, y_val)
                        , epochs=num_epochs, batch_size=100
                        , callbacks=[callback_param, checkpoint]
                        )
    return history

In [ ]:
from tensorflow import keras
labels=[0,1,2]
one_hot_vectors=keras.utils.to_categorical(labels)

x_test1=np.load('/content/drive/My Drive/MLDL_models/TESTdataset/x_test1.npy')/255.
x_test0=np.load('/content/drive/My Drive/MLDL_models/TESTdataset/x_test0.npy')/255.
x_test2=np.load('/content/drive/My Drive/MLDL_models/TESTdataset/x_test2.npy')/255.

cy_test0 = np.full((len(x_test0)), 0); cy_test0 = tf.keras.utils.to_categorical(cy_test0, num_classes=3)
cy_test1 = np.full((len(x_test1)), 1); cy_test1 = tf.keras.utils.to_categorical(cy_test1, num_classes=3)
cy_test2 = np.full((len(x_test2)), 2); cy_test2 = tf.keras.utils.to_categorical(cy_test2, num_classes=3)

x_test=np.concatenate((x_test0,x_test1,x_test2))
cy_test=np.concatenate((cy_test0,cy_test1,cy_test2))
cy_test_pred = cy_test

print(x_test.shape, cy_test.shape, cy_test_pred.shape)

(47, 128, 128, 3) (47, 3) (47, 3)


In [ ]:
sy_test1=np.load('/content/drive/My Drive/MLDL_models/TESTdataset/y_test1.npy')
sy_test0=np.load('/content/drive/My Drive/MLDL_models/TESTdataset/y_test0.npy')
sy_test2=np.load('/content/drive/My Drive/MLDL_models/TESTdataset/y_test2.npy')

sy_test=np.concatenate((sy_test0,sy_test1,sy_test2))

In [ ]:
model_input = Input((128,128,3))

In [ ]:
# Build U-Net model
def Unet(model_input):
  num_woundtypes=3
  activationFnc = 'elu'

  c1 = Conv2D(16, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (model_input)
  c1 = Dropout(0.1) (c1)
  c1 = Conv2D(16, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c1)
  p1 = MaxPooling2D((2, 2)) (c1)

  c2 = Conv2D(32, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p1)
  c2 = Dropout(0.1) (c2)
  c2 = Conv2D(32, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c2)
  p2 = MaxPooling2D((2, 2)) (c2)

  c3 = Conv2D(64, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p2)
  c3 = Dropout(0.2) (c3)
  c3 = Conv2D(64, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c3)
  p3 = MaxPooling2D((2, 2)) (c3)

  c4 = Conv2D(128, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p3)
  c4 = Dropout(0.2) (c4)
  c4 = Conv2D(128, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c4)
  p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

  c5 = Conv2D(256, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p4)
  c5 = Dropout(0.3) (c5)
  c5 = Conv2D(256, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c5)

  u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
  u6 = concatenate([u6, c4])
  c6 = Conv2D(128, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (u6)
  c6 = Dropout(0.2) (c6)
  c6 = Conv2D(128, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c6)

  u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
  u7 = concatenate([u7, c3])
  c7 = Conv2D(64, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (u7)
  c7 = Dropout(0.2) (c7)
  c7 = Conv2D(64, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c7)

  u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(32, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (u8)
  c8 = Dropout(0.1) (c8)
  c8 = Conv2D(32, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c8)

  u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
  u9 = concatenate([u9, c1], axis=3)
  c9 = Conv2D(16, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (u9)
  c9 = Dropout(0.1) (c9)
  c9 = Conv2D(16, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c9)

  so = Conv2D(1, 1, activation='sigmoid', name='so') (c9)
  w1= Flatten(input_shape=(128,128,3))(c9)
  co = Dense(num_woundtypes, activation='softmax', name='co')(w1)

  model = Model(inputs=[model_input], outputs=[co, so], name='Unet')

  return model
'''
model.compile(optimizer='adam'
              , loss='categorical_crossentropy'
              , metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]) #metric으로 accuracy + precision&recall 사용, 논문과 비교

model.summary()
'''

"\nmodel.compile(optimizer='adam'\n              , loss='categorical_crossentropy'\n              , metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]) #metric으로 accuracy + precision&recall 사용, 논문과 비교\n\nmodel.summary()\n"

In [ ]:
def conv_pool_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(3, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    return model

In [ ]:
def halfUnet(model_input):
  num_woundtypes=3
  activationFnc = 'elu'

  c1 = Conv2D(16, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (model_input)
  c1 = Dropout(0.1) (c1)
  c1 = Conv2D(16, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c1)
  p1 = MaxPooling2D((2, 2)) (c1)

  c2 = Conv2D(32, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p1)
  c2 = Dropout(0.1) (c2)
  c2 = Conv2D(32, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c2)
  p2 = MaxPooling2D((2, 2)) (c2)

  c3 = Conv2D(64, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p2)
  c3 = Dropout(0.2) (c3)
  c3 = Conv2D(64, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c3)
  p3 = MaxPooling2D((2, 2)) (c3)

  c4 = Conv2D(128, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p3)
  c4 = Dropout(0.2) (c4)
  c4 = Conv2D(128, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c4)
  p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

  c5 = Conv2D(256, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (p4)
  c5 = Dropout(0.3) (c5)
  c5 = Conv2D(256, (3, 3), activation=activationFnc, kernel_initializer='he_normal', padding='same') (c5)

  #woundtype은 c9을 Flatten
  #w1= Flatten(input_shape=(128,128,3))(c5)
  c5 = Conv2D(192, (3, 3), activation='relu', padding = 'same')(c5)
  c5 = Conv2D(192, (1, 1), activation='relu')(c5)
  c5 = Conv2D(3, (1, 1))(c5)
  c5 = GlobalAveragePooling2D()(c5)
  c5 = Activation(activation='softmax')(c5)

  model = Model(inputs=[model_input], outputs=[c5], name='halfUnet')
  return model

In [ ]:
def thesis(model_input):
  x = Conv2D(16, (31, 31),strides=(2,2), activation='relu')(model_input)
  x = Dropout(0.1)(x)
  x = Conv2D(32, (5, 5), strides=(2,2),activation='relu')(x)
  x = Dropout(0.1)(x)
  x = Conv2D(64, (5, 5),strides=(2,2), activation='relu')(x)
  x = Flatten()(x)
  x = Dense(9, activation='relu')(x)
  x = Dense(3, activation='softmax')(x)

  model = Model(inputs=[model_input], outputs=[x], name='thesis')

  return model


In [ ]:
def Densenet201(model_input):
  base_model=keras.applications.densenet.DenseNet201(include_top=False, weights='imagenet', input_tensor=model_input, input_shape=(128,128,3), pooling=None, classes=3)
  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=BatchNormalization()(x)
  x=Dropout(0.5)(x)
  x=Dense(1024,activation='relu')(x)
  x=Dense(512,activation='relu')(x)
  x=BatchNormalization()(x)
  x=Dropout(0.5)(x)

  preds=Dense(3, activation='softmax')(x)

  model = Model(base_model.input, preds, name = 'Densenet201')
  return model

In [ ]:
 def VGG19(model_input):
  base_model=keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=model_input, input_shape=(128,128,3), pooling=None, classes=3)
  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=BatchNormalization()(x)
  x=Dropout(0.5)(x)
  x=Dense(1024,activation='relu')(x)
  x=Dense(512,activation='relu')(x)
  x=BatchNormalization()(x)
  x=Dropout(0.5)(x)

  preds=Dense(3, activation='softmax')(x)

  model = Model(base_model.input, preds, name = 'VGG19')
  return model

In [ ]:
def showfigs(history):
  fig, ax = plt.subplots(2, 2, figsize=(10, 7))

  ax[0, 0].set_title('loss')
  ax[0, 0].plot(history.history['loss'], 'r')
  ax[0, 1].set_title('accuracy')
  ax[0, 1].plot(history.history['acc'], 'b')

  ax[1, 0].set_title('val_loss')
  ax[1, 0].plot(history.history['val_loss'], 'r--')
  ax[1, 1].set_title('val_accuracy')
  ax[1, 1].plot(history.history['val_acc'], 'b--')

In [ ]:
def evaluate_error(model):
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]    
    return error

In [ ]:
def predictx(model):
  example_conv=model.predict(x_test, batch_size=32)
  count=0
  for i,pred in enumerate(example_conv):
    a=np.argmax(y_test[i]); b=np.argmax(pred)
    print('(%s, %s)' % (a, b), end=' ')
    if a==b:
      count+=1
    if i == len(example_conv):
      print()
      
  print('%.2f' % (count/len(example_conv)))

In [ ]:
def evaluate_model(model):
  callback_param = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto', min_lr=1e-05)
  print('Combine->Granulation->Slough->Necrosis')
  c = model.evaluate(x_test,y_test, batch_size=10, callbacks=[callback_param])
  g = model.evaluate(x_test0, y_test0, batch_size=10, callbacks=[callback_param])
  s = model.evaluate(x_test1, y_test1, batch_size=10, callbacks=[callback_param])
  n = model.evaluate(x_test2, y_test2, batch_size=10, callbacks=[callback_param])


In [ ]:
def evaluate_model_mo(model):
  callback_param = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto', min_lr=1e-05)
  print('Combine->Granulation->Slough->Necrosis')
  c = model.evaluate(x_test, {'co' : cy_test, 'so' : sy_test}, batch_size=10, callbacks=[callback_param])
  g = model.evaluate(x_test0, {'co' : cy_test0, 'so' : sy_test0}, batch_size=10, callbacks=[callback_param])
  s = model.evaluate(x_test1, {'co' : cy_test1, 'so' : sy_test1}, batch_size=10, callbacks=[callback_param])
  n = model.evaluate(x_test2, {'co' : cy_test2, 'so' : sy_test2}, batch_size=10, callbacks=[callback_param])

In [ ]:
unet_model= Unet(model_input)
unet_model.compile(loss={'co' : 'categorical_crossentropy', 'so' : 'binary_crossentropy'}, optimizer='adam', metrics=['acc']) 
filepath = '/content/drive/MyDrive/MLDL_models/weights2/' + unet_model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', save_freq=10)
callback_param = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto', min_lr=1e-05)
history = unet_model.fit(x_train
                        , {'co' : cy_train, 'so' : sy_train}
                        , validation_data=(x_val, {'co' : cy_val, 'so' : sy_val})
                        , epochs=100, batch_size=100
                        , callbacks=[callback_param, checkpoint]
                        )

In [ ]:
classpreds, segpreds = unet_model.predict(x_test, batch_size=10)

In [ ]:
def realMIOU(y_test, preds):
  preds[preds>=0.5] = 1
  preds[preds<0.5]=0
  m=tf.keras.metrics.MeanIoU(num_classes=2)
  m.update_state(y_test,preds)
  print(m.result())

realMIOU(sy_test, segpreds)

tf.Tensor(0.4108198, shape=(), dtype=float32)


In [ ]:
evaluate_model_mo(unet_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 8ms/step - loss: 2.3847 - co_loss: 1.8981 - so_loss: 0.4866 - co_acc: 0.6667 - so_acc: 0.7687


In [ ]:
evaluate_model_mo(unet_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 10ms/step - loss: 5.1481 - co_loss: 4.6742 - so_loss: 0.4739 - co_acc: 0.1429 - so_acc: 0.7720


In [ ]:
unet_model=Unet(model_input)
unet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m])
unet_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/Unet.43-0.22.hdf5')
evaluate_model(unet_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 6ms/step - loss: 4.4832 - acc: 0.1429 - precision_m: 0.4000 - recall_m: 0.4000


In [ ]:
halfunet_model=halfUnet(model_input)
halfunet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m])
halfunet_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/halfUnet.48-0.23.hdf5')
evaluate_model(halfunet_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 4ms/step - loss: 1.4627 - acc: 0.4762 - precision_m: 0.6481 - recall_m: 0.6333


In [ ]:
conv_pool_cnn_model=conv_pool_cnn(model_input)
conv_pool_cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m]) 
conv_pool_cnn_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/conv_pool_cnn.50-0.48.hdf5')
evaluate_model(conv_pool_cnn_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 74ms/step - loss: 2.9039 - acc: 0.3810 - precision_m: 0.5750 - recall_m: 0.5333


In [ ]:
dense_model = Densenet201(model_input)
dense_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m]) 
dense_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/functional_1.44-0.17.hdf5')
evaluate_model(dense_model)

In [ ]:
vgg_model = VGG19(model_input)
vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m]) 
vgg_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/VGG19.49-0.45.hdf5')
evaluate_model(vgg_model)

In [ ]:
#@title 여기부터 앙상블 관련 코드입니다


In [ ]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [ ]:
def weighted_ensemble(models, model_input, weights):
    
    outputs = [model.outputs[0]*weights[i] for i,model in enumerate(models)]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [ ]:
#metrics
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
halfunet_model = halfUnet(model_input)
unet_model = Unet(model_input)

conv_pool_cnn_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/conv_pool_cnn.99-0.23.hdf5')
halfunet_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/halfUnet.48-0.23.hdf5')
unet_model.load_weights('/content/drive/MyDrive/MLDL_models/weights/Unet.13-0.70.hdf5')

models = [unet_model, halfunet_model, conv_pool_cnn_model]
models2 = [unet_model, halfunet_model]



In [ ]:
def ensemble(model_input):
    
    outputs = [unet_model.outputs[0], halfunet_model.outputs[0]]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [ ]:
ensemble_model = ensemble(model_input)
ensemble_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m]) 

ValueError: ignored

In [ ]:
weights = [0.7,1,0.3]
w_ensemble_model = weighted_ensemble(models, model_input, weights)
w_ensemble_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', precision_m, recall_m]) 

In [ ]:
#0.7,1
evaluate_model(w_ensemble_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 8ms/step - loss: 1.8465 - acc: 0.4286 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00


In [ ]:
evaluate_model(w_ensemble_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 16ms/step - loss: 1.7505 - acc: 0.3333 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00


In [ ]:
evaluate_model(ensemble_model)

Combine->Granulation->Slough->Necrosis
3/3 [==============================] - 0s 15ms/step - loss: 1.9113 - acc: 0.1905 - precision_m: 0.4370 - recall_m: 0.4000


In [ ]:
#@title 이하 사용하지 않는 코드입니다